In [1]:
from pathlib import Path

import geopandas as gpd

import grid_tools as gt
from tii_blends import tiled_blending

## Select file(s)

List paths to files. Paths can be passed as plain string format or as a `pathlib.Path` object.

In [2]:
# List of INPUT DATASETS (have to be in this format)
list_tifs = [
    Path(r".\test_data\test_small.tif")
]

## Select blends

Select the desired blends by listing the keywords:

```
blend_types = ["SLRM", "vat_combined_8bit"]
```

Choose from the following keywords for blends:
* 'vat_combined_8bit'
* 'e2MSTP'
* 'e4MSTP'

Keywords for basic visualizations:
* "SLRM"
* "hs"
* "svf"

> All visualizations are saved as 8 bit images using the default values for stretch and cut off

In [3]:
# Select visualizations
blend_types = [
    "SLRM",
    "vat_combined_8bit"
]

## Other inputs

the followinf parameters are self explanatory.

In [4]:
# CONSTANT INPUT PARAMETERS

tile_size = 1000  # in PIXELS!

refgrid_existing = False  # Change to true, to avoid re-creating refgrid

nr_processes = 3  # How many CPUs to use - deoends on your PC

## Run the main routine

Start processing by executing the following cell:

In [5]:
for in_file in list_tifs:

    # (1b) Instead of folder, the tif/vrt is given
    input_vrt = in_file.as_posix()
    ds = in_file.parent

    print("Start --- " + ds.name)

    # (2) and (3) if refgrid doesn't exist, else read it from file
    if refgrid_existing:
        # I have added this option for Noise Mapping, because vrt and refgrid are filtered to contain
        # only the tiles with archeology.
        print("* refgrid exists; Reading from file...")
        refgrid_name = list(ds.glob("*_refgrid*.gpkg"))[0]
        tiles_extents = gpd.read_file(refgrid_name)
    else:
        # (2) To filter we need polygon covering valid data
        vdm_file = list(ds.glob("*_validDataMask*"))

        if vdm_file:
            print("* validDataMask exists; Removing and creating new...")
            Path(vdm_file[0]).unlink()
        else:
            # If it doesn't exist, try creating from VRT
            print("* validDataMask doesn't exists; Creating...")

        valid_data_outline = gt.poly_from_valid(input_vrt, save_gpkg=True)

        # (3) Create reference grid, filter it and save it to disk
        print("Create REF GRID")

        refgrid_name = input_vrt[:-4] + "_refgrid.gpkg"
        if Path(refgrid_name).exists():
            Path(refgrid_name).unlink()

        tiles_extents = gt.bounding_grid(input_vrt, tile_size, tag=True)
        tiles_extents = gt.filter_by_outline(
            tiles_extents, valid_data_outline.as_posix(),
            save_gpkg=True, save_path=refgrid_name
        )

    # Run tiled blending (visualizations calculated on the go, stored in memory)
    tiles_list = tiles_extents[["minx", "miny", "maxx", "maxy"]].values.tolist()

    tiled_blending(
        blend_types=blend_types,
        input_vrt_path=in_file,
        tiles_list=tiles_list,
        nr_processes=nr_processes
    )

Start --- test_data
* validDataMask doesn't exists; Creating...
Create REF GRID
Finished tile: 243500_168500
Finished tile: 244000_168500
Finished tile: 244500_168500
Finished tile: 243500_169000
Finished tile: 244000_169000
Finished tile: 244500_169000
Finished tile: 243500_169500
Finished tile: 244000_169500
Finished tile: 244500_169500
  - Created: test_data\test_small_vat_combined_8bit.vrt
Done with computing blends in 1 min.
